In [34]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import (
    concat,
    count,
    col,
    from_csv,
    lit,
    count,
    max,
    first,
    struct,
    first,
    last,
    unix_timestamp,
    avg
)

spark = SparkSession.builder.appName("EsempioStatico").getOrCreate()

In [22]:
options = {"sep": ","}
schema = "targa INT, varco INT, corsia DOUBLE, timestamp TIMESTAMP, nazione STRING"

df = spark.read.option("header",True) \
    .schema(schema) \
    .csv("test.csv")

df.printSchema()
df.show()

root
 |-- targa: integer (nullable = true)
 |-- varco: integer (nullable = true)
 |-- corsia: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- nazione: string (nullable = true)

+-------+-----+------+-------------------+-------+
|  targa|varco|corsia|          timestamp|nazione|
+-------+-----+------+-------------------+-------+
| 515364|   25|   2.0|2022-11-05 00:00:00|      I|
| 199201|   27|   1.0|2022-11-05 00:00:01|      I|
|3053187|    2|   1.0|2022-11-05 00:00:02|      H|
|4156088|    2|   1.0|2022-11-05 00:00:03|     RO|
| 515364|   20|   2.0|2022-11-05 00:00:30|      I|
| 199201|    9|   1.0|2022-11-05 00:00:31|      I|
|3053187|   16|   1.0|2022-11-05 00:00:32|      H|
|4156088|   16|   1.0|2022-11-05 00:00:33|     RO|
| 515364|    2|   2.0|2022-11-05 00:01:00|      I|
| 199201|   26|   1.0|2022-11-05 00:01:01|      I|
|3053187|    4|   1.0|2022-11-05 00:01:02|      H|
|4156088|    4|   1.0|2022-11-05 00:01:03|     RO|
| 515364|   16|   2.0|2022-11-05

In [3]:
# Immette i tratti autostradali
tratti = [
    (27, 9, 8.48),
    (9, 26, 17.42),
    (26, 10, 6.0),
    (10, 18, 12.3),
    (18, 23, 14.0),
    (23, 15, 17.6),
    (15, 5, 7.7),
    (5, 8, 10.9),
    (8, 3, 6.9),
    (3, 13, 9.8),
    (22, 1, 10.6),
    (1, 12, 10.9),
    (12, 25, 7.7),
    (25, 20, 17.7),
    (20, 2, 13.8),
    (2, 16, 14.1),
    (16, 4, 14.0),
    (4, 21, 25.7),
]

tratti_schema = StructType(
    [
        StructField("ingresso", IntegerType()),
        StructField("uscita", IntegerType()),
        StructField("lunghezza", DoubleType())
    ]
)

df_tratti = spark.createDataFrame(data=tratti, schema=tratti_schema).cache()
df_tratti.show()

+------+--------+------+---------+
|tratto|ingresso|uscita|lunghezza|
+------+--------+------+---------+
|     1|      27|     9|     8.48|
|     2|       9|    26|    17.42|
|     3|      26|    10|      6.0|
|     4|      10|    18|     12.3|
|     5|      18|    23|     14.0|
|     6|      23|    15|     17.6|
|     7|      15|     5|      7.7|
|     8|       5|     8|     10.9|
|     9|       8|     3|      6.9|
|    10|       3|    13|      9.8|
|    11|      22|     1|     10.6|
|    12|       1|    12|     10.9|
|    13|      12|    25|      7.7|
|    14|      25|    20|     17.7|
|    15|      20|     2|     13.8|
|    16|       2|    16|     14.1|
|    17|      16|     4|     14.0|
|    18|       4|    21|     25.7|
+------+--------+------+---------+



In [4]:
df_left = df.join(df_tratti, (df.varco == df_tratti.ingresso ) | (df.varco == df_tratti.uscita), 'left')
df_left.show()

+-------+-----+------+-------------------+-------+------+--------+------+---------+
|  targa|varco|corsia|          timestamp|nazione|tratto|ingresso|uscita|lunghezza|
+-------+-----+------+-------------------+-------+------+--------+------+---------+
| 515364|   25|   2.0|2022-11-05 00:00:00|      I|    13|      12|    25|      7.7|
| 515364|   25|   2.0|2022-11-05 00:00:00|      I|    14|      25|    20|     17.7|
| 199201|   27|   1.0|2022-11-05 00:00:01|      I|     1|      27|     9|     8.48|
|3053187|    2|   1.0|2022-11-05 00:00:02|      H|    15|      20|     2|     13.8|
|3053187|    2|   1.0|2022-11-05 00:00:02|      H|    16|       2|    16|     14.1|
|4156088|    2|   1.0|2022-11-05 00:00:03|     RO|    15|      20|     2|     13.8|
|4156088|    2|   1.0|2022-11-05 00:00:03|     RO|    16|       2|    16|     14.1|
| 515364|   20|   2.0|2022-11-05 00:00:30|      I|    14|      25|    20|     17.7|
| 515364|   20|   2.0|2022-11-05 00:00:30|      I|    15|      20|     2|   

In [54]:
df_aggregated = df_left.groupBy(['targa', 'ingresso', 'uscita']) \
    .agg(
        first('timestamp').alias('partenza'), 
        last('timestamp').alias('arrivo'),
        count('timestamp').alias('avvistamenti')
    )
df_aggregated.show()

+-------+--------+------+-------------------+-------------------+------------+
|  targa|ingresso|uscita|           partenza|             arrivo|avvistamenti|
+-------+--------+------+-------------------+-------------------+------------+
|3053187|      20|     2|2022-11-05 00:00:02|2022-11-05 00:00:02|           1|
|3053187|      16|     4|2022-11-05 00:00:32|2022-11-05 00:01:02|           2|
|4156088|       4|    21|2022-11-05 00:01:03|2022-11-05 00:01:33|           2|
|4156088|       2|    16|2022-11-05 00:00:03|2022-11-05 00:00:33|           2|
| 199201|      26|    10|2022-11-05 00:01:01|2022-11-05 00:01:31|           2|
|4156088|      20|     2|2022-11-05 00:00:03|2022-11-05 00:00:03|           1|
| 515364|      25|    20|2022-11-05 00:00:00|2022-11-05 00:00:30|           2|
| 515364|      20|     2|2022-11-05 00:00:30|2022-11-05 00:01:00|           2|
| 515364|       2|    16|2022-11-05 00:01:00|2022-11-05 00:01:30|           2|
| 199201|       9|    26|2022-11-05 00:00:31|2022-11

Dopo questa aggregazione si possono ramificare processi diversi, ad esempio per ottenere il numero di targhe per tratto si può raggruppare per tratto (ingresso, uscita) ed eseguire il ``count()``. 

Invece, per ottenere la velocità della targa, su un tratto si rifà il join con la tabella tratti per avere anche la distanza, e si crea una nuova colonna che calcola: ``lunghezza / (arrivo - partenza)``. Da qui, se si vuole l'ultimo tratto di una targa: 
1. raggruppa per targa
2. ordina per timestamp
3. aggrega per ultima riga, ``last()``

Oppure si calcola la velocità media della targa:
1. raggruppa per targa
2. aggrega con ``avg()``

In tutti i casi, bisogna prima filtrare via le righe con ``avvistamenti = 2``. Questo perché le righe con ``avvistamenti = 1`` rappresentano tutte quelle targhe che sono state avvistate solo ad un varco e non al successivo, perché magari sono uscite o il sensore non le ha rilevate correttamente.

 ## Velocità ultimo avvistamento per targa

In [59]:
df_speed = df_aggregated \
     .join(df_tratti, (df_aggregated.ingresso == df_tratti.ingresso ), 'left') \
     .filter(col('avvistamenti') == 2) \
     .orderBy('partenza') \
     .groupBy(df_aggregated.targa) \
     .agg(
          last('arrivo').alias('arrivo'),
          last('partenza').alias('partenza'),
          last('lunghezza').alias('lunghezza'),
          last(df_aggregated.ingresso).alias('ingresso'),
          last(df_aggregated.uscita).alias('uscita')
     ) \
     .withColumn('velocità', \
     ( (col('lunghezza') * 1000) / (unix_timestamp(col('arrivo')) - unix_timestamp(col('partenza')))) * 3.6) \
     .select('targa', col('arrivo').alias('timestamp'), 'ingresso', 'uscita', 'velocità')

df_speed.show()

+-------+-------------------+--------+------+--------+
|  targa|          timestamp|ingresso|uscita|velocità|
+-------+-------------------+--------+------+--------+
| 199201|2022-11-05 00:01:31|      26|    10|   720.0|
|4156088|2022-11-05 00:01:33|       4|    21|  3084.0|
|3053187|2022-11-05 00:01:32|       4|    21|  3084.0|
| 515364|2022-11-05 00:01:30|       2|    16|  1692.0|
+-------+-------------------+--------+------+--------+



## Velocità media per targa

In [46]:
df_average = df_aggregated \
    .join(df_tratti, (df_aggregated.ingresso == df_tratti.ingresso ), 'left') \
    .filter(col('avvistamenti') == 2) \
    .orderBy('partenza') \
    .withColumn('velocità', \
    ( (col('lunghezza') * 1000) / (unix_timestamp(col('arrivo')) - unix_timestamp(col('partenza')))) * 3.6) \
    .groupBy('targa') \
    .agg(
        last(df_aggregated.ingresso).alias('ingresso'),
        last(df_aggregated.uscita).alias('uscita'),
        avg('velocità').alias('velocità_media')
    ) \
    .select('targa', 'ingresso', 'uscita', 'velocità_media')

df_average.show()

+-------+--------+------+--------------+
|  targa|ingresso|uscita|velocità_media|
+-------+--------+------+--------------+
| 199201|      26|    10|        1276.0|
|4156088|       4|    21|        2152.0|
|3053187|       4|    21|        2152.0|
| 515364|       2|    16|        1824.0|
+-------+--------+------+--------------+



## Conteggio targhe per tratto

Ha bisogno di altre due aggregazioni: una per ottenere solo una targa (come in velocità puntutale) e l'altra raggruppando per tratto.

In [64]:
df_single = df_aggregated \
     .join(df_tratti, (df_aggregated.ingresso == df_tratti.ingresso ), 'left') \
     .filter(col('avvistamenti') == 2) \
     .orderBy('partenza') \
     .groupBy(df_aggregated.targa) \
     .agg(
          last(df_aggregated.ingresso).alias('ingresso'),
          last(df_aggregated.uscita).alias('uscita')
     ) 
    

df_count = df_single \
     .groupBy('ingresso', 'uscita') \
     .agg(count('targa').alias('conteggio')) 

df_count.show()

+--------+------+---------+
|ingresso|uscita|conteggio|
+--------+------+---------+
|       4|    21|        2|
|      26|    10|        1|
|       2|    16|        1|
+--------+------+---------+

